In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from textblob import TextBlob
import plotly.graph_objects as go
import plotly.express as px

In [7]:
trump = pd.read_csv(r'C:\Users\amany\Desktop\archive datasets\nlpelection\Trumpall2.csv')
biden = pd.read_csv(r'C:\Users\amany\Desktop\archive datasets\nlpelection\Bidenall2.csv')
trump.shape,biden.shape

((2788, 2), (2540, 2))

In [8]:
trump.head()

,user,text
0,manny_rosen,@sanofi please tell us how many shares the Cr...
1,osi_abdul,"https://t.co/atM98CpqF7 Like, comment, RT #P..."
2,Patsyrw,Your AG Barr is as useless &amp; corrupt as y...
3,seyedebrahimi_m,Mr. Trump! Wake Up! Most of the comments bel...
4,James09254677,After 4 years you think you would have figure...


In [9]:
biden.head()

,user,text
0,MarkHodder3,@JoeBiden And we’ll find out who won in 2026...
1,K87327961G,@JoeBiden Your Democratic Nazi Party cannot be...
2,OldlaceA,@JoeBiden So did Lying Barr
3,penblogger,@JoeBiden It's clear you didnt compose this tw...
4,Aquarian0264,@JoeBiden I will vote in person thank you.


In [13]:
print(biden.isnull().sum(),"\n\n",trump.isnull().sum())

user    0
text    0
dtype: int64 

 user    0
text    0
dtype: int64


# Sentiment Analysis
Now, I will get started with sentiment analysis. I will do it by using the Textblob package in Python. Here I will use this package to perform simple text classification in either positive or negative on the basis of sentiment analysis:

In [21]:
trump_sent=TextBlob(trump['text'][10])
print(trump_sent)
print("Trump Sentiment : ",trump_sent.sentiment,"\n\n")

biden_sent=TextBlob(biden['text'][699])
print(biden_sent)
print("Trump Sentiment : ",biden_sent.sentiment)

 No president would be able to destroy as you have
Trump Sentiment :  Sentiment(polarity=0.15, subjectivity=0.3125) 


@JoeBiden We need someone who has his own thoughts not someone reading and sending someone else's tweets.
Trump Sentiment :  Sentiment(polarity=0.6, subjectivity=1.0)


In [22]:
def senti(review):
    return TextBlob(review).sentiment.polarity
trump['sentiment_polarity']=trump['text'].apply(senti)
biden['sentiment_polarity']=biden['text'].apply(senti)

In [23]:
trump.head()

,user,text,sentiment_polarity
0,manny_rosen,@sanofi please tell us how many shares the Cr...,0.05
1,osi_abdul,"https://t.co/atM98CpqF7 Like, comment, RT #P...",0.00
2,Patsyrw,Your AG Barr is as useless &amp; corrupt as y...,-0.50
3,seyedebrahimi_m,Mr. Trump! Wake Up! Most of the comments bel...,0.50
4,James09254677,After 4 years you think you would have figure...,0.00


In [24]:
biden.head()

,user,text,sentiment_polarity
0,MarkHodder3,@JoeBiden And we’ll find out who won in 2026...,0.00
1,K87327961G,@JoeBiden Your Democratic Nazi Party cannot be...,0.00
2,OldlaceA,@JoeBiden So did Lying Barr,0.00
3,penblogger,@JoeBiden It's clear you didnt compose this tw...,0.05
4,Aquarian0264,@JoeBiden I will vote in person thank you.,0.00


### Sentiment Polarity on Both the candidates:
Now I will add a new attribute in both the datasets by the name of “Expression Label”:

In [25]:
trump['Expression Label']=np.where(trump['sentiment_polarity']>0,'positive','negative')
trump['Expression Label'][trump['sentiment_polarity']==0]='neutral'
trump.tail()

F:\anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,user,text,sentiment_polarity,Expression Label
2783,4diva63,"@realDonaldTrump For the 1/100 time, absentee ...",0.000,neutral
2784,hidge826,@realDonaldTrump If you’re so scared of losing...,0.000,neutral
2785,SpencerRossy,@realDonaldTrump I rarely get involved with fo...,0.225,positive
2786,ScoobyMcpherson,@realDonaldTrump This is the moment when Trump...,0.000,neutral
2787,bjklinz,"@realDonaldTrump I’m sorry, Donald. No. #POTUS",-0.500,negative


In [26]:
biden['Expression Label']=np.where(biden['sentiment_polarity']>0,'positive','negative')
biden['Expression Label'][biden['sentiment_polarity']==0]='neutral'
biden.tail()

F:\anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,user,text,sentiment_polarity,Expression Label
2535,meryn1977,@JoeBiden You'll just try to calm those waters...,0.15,positive
2536,BSNelson114,@JoeBiden 96 days 96 dias #VoteJoeBiden2020 #...,0.00,neutral
2537,KenCapel,@JoeBiden YOU THINK YOU CAN DO THAT??? YOU CAN...,0.00,neutral
2538,LeslyeHale,@JoeBiden Trump wants our children back at sch...,0.10,positive
2539,rerickre,"@JoeBiden ... and I know, because it’s much co...",0.20,positive


Now I will drop all the tweets with neutral polarity from both the datasets to balance the data equally. I will also perform some data cleaning operations so that at the can we can easily predict the US Elections:

In [29]:
reviews1=trump[trump['sentiment_polarity']==0.0000]
print(reviews1.shape)

cond1=trump['sentiment_polarity'].isin(reviews1['sentiment_polarity'])
cond1

(1464, 4)


0       False
1        True
2       False
3       False
4        True
        ...  
2783     True
2784     True
2785    False
2786     True
2787    False
Name: sentiment_polarity, Length: 2788, dtype: bool

In [30]:
trump[cond1].index

Int64Index([   1,    4,    5,    7,    9,   11,   12,   14,   15,   16,
            ...
            2775, 2776, 2777, 2778, 2779, 2780, 2781, 2783, 2784, 2786],
           dtype='int64', length=1464)

In [31]:
trump.drop(trump[cond1].index, inplace = True)
trump.shape

(1324, 4)

In [32]:
reviews2=biden[biden['sentiment_polarity']==0.0000]
print(reviews2.shape)

cond2=biden['sentiment_polarity'].isin(reviews2['sentiment_polarity'])
biden.drop(biden[cond2].index, inplace = True)
biden.shape

(1509, 4)


(1031, 4)

#### Now, before moving forward we need to balance both the datasets:

In [33]:
# Donald Trump
np.random.seed(10)
remove_n =324
drop_indices = np.random.choice(trump.index, remove_n, replace=False)
df_subset_trump = trump.drop(drop_indices)
print(df_subset_trump.shape)


# Joe Biden
np.random.seed(10)
remove_n =31
drop_indices = np.random.choice(biden.index, remove_n, replace=False)
df_subset_biden = biden.drop(drop_indices)
print(df_subset_biden.shape)

(1000, 4)
(1000, 4)


#### Now let’s analyze the data to predict the US Elections, by analyzing the number of positive and negative sentiments in both the accounts:

In [46]:
count1=df_subset_trump.groupby('Expression Label').count()
print(count1,"\n\n")

negative_per1 = (count1['sentiment_polarity'][0]/1000)*100
positive_per1 = (count1['sentiment_polarity'][1]/1000)*100
print(negative_per1,positive_per1)

                  user  text  sentiment_polarity
Expression Label                                
negative           449   449                 449
positive           551   551                 551 


44.9 55.1


In [47]:
count2=df_subset_biden.groupby('Expression Label').count()
print(count2,"\n\n")

negative_per2 = (count2['sentiment_polarity'][0]/1000)*100
positive_per2 = (count2['sentiment_polarity'][1]/1000)*100
print(negative_per2,positive_per2)

                  user  text  sentiment_polarity
Expression Label                                
negative           393   393                 393
positive           607   607                 607 


39.300000000000004 60.699999999999996


In [50]:
Politicians = ['Donald Trump','Joe Biden']
lis_pos = [positive_per1, positive_per2]
lis_neg = [negative_per1, negative_per2]

In [52]:
fig = go.Figure(data=[
    go.Bar(name='Positive', x=Politicians, y=lis_pos),
    go.Bar(name='Negative', x=Politicians, y=lis_neg)
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()